In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [2]:
XX_coefficient=pd.read_csv('XX_coefficient.csv')
XY_coefficient=pd.read_csv('XY_coefficient_2.csv')
YY_coefficient=pd.read_csv('YY_coefficient.csv')
bio_feature=pd.read_csv('bio_feature_test.csv')
label_one_hot=pd.read_csv('label_one_hot_test.csv')

In [3]:
224+97

321

In [4]:
XY_coefficient.values.shape

(224, 97)

In [5]:
YY_new=[]
for s in range(97):
    tmp=[]
    for t in range(97):
        numerator=0
        denominator_s=0
        denominator_t=0
        mu_s=sum(XY_coefficient.values[:,s])
        mu_t=sum(XY_coefficient.values[:,t])
        for g in range(224):
            numerator+=(XY_coefficient.values[g,s]-mu_s)*(XY_coefficient.values[g,t]-mu_t)
            denominator_s+=(XY_coefficient.values[g,s]-mu_s)**2
            denominator_t+=(XY_coefficient.values[g,t]-mu_t)**2
        tmp.append(numerator/(np.sqrt(denominator_s)*np.sqrt(denominator_t)))
    YY_new.append(tmp)

In [6]:
YY_new

[[0.9999999999999998,
  0.9546701313526628,
  0.9603436145027949,
  -0.9582763345601109,
  -0.9626850313803967,
  0.9604580172775852,
  -0.9618564920629542,
  0.9587451237409186,
  -0.9597876460423633,
  -0.633373271212058,
  0.9917697716948508,
  0.9953842253703478,
  -0.9627410380509306,
  -0.9620852502850635,
  0.9576244641023031,
  0.9592881101559896,
  -0.9618386548475346,
  -0.8113478867078392,
  0.9399206550166479,
  0.9609421012759554,
  0.9571873922171253,
  0.9575457548315454,
  0.9598710432192338,
  -0.962087222046144,
  -0.9597546837702771,
  -0.9606944746359891,
  0.9289525891265002,
  0.8915644841152887,
  0.9605080954723955,
  0.9584361953344259,
  0.9589526513113534,
  -0.9439696683545654,
  -0.9578653115021428,
  0.844400504070655,
  -0.9603028496414452,
  -0.9422607786686955,
  0.9503491986427193,
  0.9602369219774651,
  -0.9593528023191126,
  -0.9589568247861371,
  -0.9599086929883719,
  -0.34196449137011437,
  -0.7353921500639569,
  0.9927329456695709,
  0.947549901

In [7]:
coefficient_matrix1=np.concatenate((XX_coefficient.values,XY_coefficient.values),1)
coefficient_matrix2=np.concatenate((XY_coefficient.values.T,np.array(YY_new)),1)

In [8]:
coefficient_matrix=np.concatenate((coefficient_matrix1,coefficient_matrix2))

In [9]:
coefficient_matrix.shape

(321, 321)

In [10]:
np.all(np.linalg.eigvals(coefficient_matrix) >= 0)

False

In [11]:
n=0
while np.all(np.linalg.eigvals(coefficient_matrix) >= 0)==False:
    coefficient_matrix=coefficient_matrix*0.999
    np.fill_diagonal(coefficient_matrix, 1)
    n+=1


In [12]:
print(n)

2260


In [13]:
np.all(np.linalg.eigvals(coefficient_matrix) >= 0)

True

In [14]:
XX_coefficient_PSD=coefficient_matrix[:224,:224]

In [15]:
XY_coefficient_PSD=coefficient_matrix[:224,224:]

In [16]:
YY_coefficient_PSD=coefficient_matrix[224:,224:]

new

In [17]:

from scipy.linalg import fractional_matrix_power
from scipy.linalg import svd

In [18]:
fractional_matrix_power(XX_coefficient_PSD, -0.5)

array([[ 1.00003982e+00, -1.14104840e-03,  5.34486713e-05, ...,
        -7.88871740e-04,  1.87301062e-04, -1.15097944e-04],
       [-2.10432524e-04,  1.00054936e+00, -3.98298663e-04, ...,
        -1.32141892e-04,  1.73059539e-05,  7.38123015e-04],
       [ 4.48961373e-05, -1.81414549e-03,  1.00014007e+00, ...,
        -2.43042486e-04, -7.62993063e-04,  2.82765943e-04],
       ...,
       [-6.55982670e-04, -5.95823584e-04, -2.40600266e-04, ...,
         1.00014136e+00,  3.10769960e-04, -6.40139446e-04],
       [ 1.90447017e-05,  9.54158841e-06, -9.23596888e-05, ...,
         3.80002978e-05,  1.00066852e+00,  9.79427232e-05],
       [-4.67376874e-05,  1.62524785e-03,  1.36695649e-04, ...,
        -3.12599497e-04,  3.91145098e-04,  1.00087694e+00]])

In [19]:
fractional_matrix_power(YY_coefficient_PSD, -0.5)

array([[ 1.04497041, -0.00762943, -0.00780353, ..., -0.00787797,
        -0.00782899,  0.00781872],
       [-0.00762943,  1.04817847, -0.00814717, ..., -0.00814927,
        -0.00812687,  0.00812049],
       [-0.00780353, -0.00814717,  1.04839073, ..., -0.00817563,
        -0.00817656,  0.00814244],
       ...,
       [-0.00787797, -0.00814927, -0.00817563, ...,  1.04838724,
        -0.00818206,  0.00814642],
       [-0.00782899, -0.00812687, -0.00817656, ..., -0.00818206,
         1.04837723,  0.00814664],
       [ 0.00781872,  0.00812049,  0.00814244, ...,  0.00814642,
         0.00814664,  1.04822216]])

In [20]:
K=fractional_matrix_power(XX_coefficient_PSD, -0.5).dot(XY_coefficient_PSD).dot(np.real(fractional_matrix_power(YY_coefficient_PSD, -0.5)))

In [21]:
XX_coefficient_PSD

array([[ 1.00000000e+00,  2.32806255e-03, -1.13800217e-04, ...,
         1.65586578e-03, -3.99470422e-04,  1.86729865e-04],
       [ 4.29342067e-04,  1.00000000e+00,  8.03501538e-04, ...,
         2.72352678e-04, -1.73912650e-04, -1.41503226e-03],
       [-9.55905924e-05,  3.65973785e-03,  1.00000000e+00, ...,
         5.00610424e-04,  1.68605076e-03, -5.36480808e-04],
       ...,
       [ 1.37692758e-03,  1.22802955e-03,  4.95580024e-04, ...,
         1.00000000e+00, -6.77930400e-04,  1.40422824e-03],
       [-4.06180026e-05, -9.58862447e-05,  2.04095071e-04, ...,
        -8.28959049e-05,  1.00000000e+00, -1.88249548e-04],
       [ 7.58251776e-05, -3.11571119e-03, -2.59347329e-04, ...,
         6.85727220e-04, -7.51795392e-04,  1.00000000e+00]])

In [22]:
K

array([[ 1.62944144e-03,  9.85367348e-03,  3.77921072e-03, ...,
        -3.09879413e-04, -1.91201509e-03, -9.03213071e-05],
       [-6.94645609e-03, -3.23986200e-02, -2.19264275e-02, ...,
         2.34057721e-03,  2.08110772e-03, -9.47183873e-03],
       [-6.92823099e-03, -8.42144659e-04, -3.47033534e-03, ...,
         6.79298784e-04, -7.95908710e-05, -2.29826192e-03],
       ...,
       [-6.05910599e-03,  6.30170593e-03,  1.81281054e-03, ...,
        -1.54515762e-03, -1.79805073e-03, -7.31849497e-04],
       [-1.29922530e-02, -2.72519712e-02, -1.42044902e-02, ...,
         3.56434270e-03,  2.79956201e-03,  3.06312669e-03],
       [-1.86718841e-02,  4.87754780e-02,  1.34971078e-03, ...,
        -3.84552830e-03, -5.30286736e-03,  1.75668843e-03]])

In [23]:
alpha, D, gama = svd(K)

In [24]:
alpha.shape

(224, 224)

In [25]:
alpha[:, :10].shape

(224, 10)

In [26]:
D.shape

(97,)

In [27]:
gama.T[:, :10].shape

(97, 10)

In [28]:
a=fractional_matrix_power(XX_coefficient_PSD, -0.5).dot(alpha[:, :1])

In [29]:
a.shape

(224, 1)

In [30]:
b=np.real(fractional_matrix_power(YY_coefficient_PSD, -0.5)).dot(gama.T[:, :1])

In [31]:
np.nanmax(a.T.dot(XY_coefficient_PSD).dot(b)/np.sqrt(a.T.dot(XX_coefficient_PSD).dot(a))/np.sqrt(b.T.dot(YY_coefficient_PSD).dot(b)))

0.9993568138966943

In [32]:
for i in range(10):
    a=fractional_matrix_power(XX_coefficient_PSD, -0.5).dot(alpha[:, i:i+1])
    b=np.real(fractional_matrix_power(YY_coefficient_PSD, -0.5)).dot(gama.T[:, i:i+1])
    print(a.T.dot(XY_coefficient_PSD).dot(b)/np.sqrt(a.T.dot(XX_coefficient_PSD).dot(a))/np.sqrt(b.T.dot(YY_coefficient_PSD).dot(b)))

[[0.99935681]]
[[0.63957425]]
[[0.55470908]]
[[0.34476019]]
[[0.32995782]]
[[0.24352816]]
[[0.21168672]]
[[0.17883784]]
[[0.16680713]]
[[0.15778535]]


In [33]:
a.shape

(224, 1)

In [34]:
b.shape

(97, 1)

In [35]:
label_one_hot.values.dot(b)

array([[-0.64131128],
       [ 0.36593802],
       [ 0.06327851],
       ...,
       [ 0.63500945],
       [ 1.27462965],
       [ 1.16515617]])

In [36]:
for i in range(10):
    a=fractional_matrix_power(XX_coefficient_PSD, -0.5).dot(alpha[:, i:i+1])
    b=np.real(fractional_matrix_power(YY_coefficient_PSD, -0.5)).dot(gama.T[:, i:i+1])
    print(np.corrcoef(bio_feature.values.dot(a).reshape(-1),label_one_hot.values.dot(b).reshape(-1))[0,1])

0.8959835479937306
0.9643758697609611
0.9292429935995529
0.8896619265268776
0.719970882728853
0.7674178851372238
0.7534482783021587
0.7486402889523536
0.5833090130216152
0.6097543570872871


In [37]:
a=fractional_matrix_power(XX_coefficient_PSD, -0.5).dot(alpha[:, :10])
b=np.real(fractional_matrix_power(YY_coefficient_PSD, -0.5)).dot(gama.T[:, :10])

# 

In [38]:
pd.DataFrame(bio_feature.values.dot(a)).to_csv('metaCCA_fe.csv')
pd.DataFrame(label_one_hot.values.dot(b)).to_csv('metaCCA_ta.csv')

In [39]:
a.T.dot(XY_coefficient_PSD).dot(b)/np.sqrt(a.T.dot(XX_coefficient_PSD).dot(a))/np.sqrt(b.T.dot(YY_coefficient_PSD).dot(b))

D:\anaconda\envs\zhu-python\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in sqrt
  """Entry point for launching an IPython kernel.


array([[9.99356814e-01,            nan,            nan,            nan,
                   nan,            nan,            nan,            nan,
        8.53320164e+04, 1.09109338e+05],
       [           nan, 6.39574247e-01,            nan,            nan,
        6.51027103e+04,            nan,            nan,            nan,
                   nan,            nan],
       [           nan,            nan, 5.54709079e-01,            nan,
        3.14884722e+04,            nan,            nan,            nan,
                   nan,            nan],
       [5.63039780e+05,            nan,            nan, 3.44760192e-01,
        3.97499199e+04,            nan,            nan,            nan,
        1.05128420e+05,            nan],
       [           nan,            nan,            nan,            nan,
        3.29957822e-01, 1.96791459e+05, 3.64325618e+04, 1.10477337e+05,
                   nan,            nan],
       [           nan,            nan,            nan,            nan,
   

In [40]:
a.T.dot(XY_coefficient).dot(b).shape

(10, 10)

In [41]:
np.sqrt(a.T.dot(XX_coefficient).dot(a)).shape

D:\anaconda\envs\zhu-python\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in sqrt
  """Entry point for launching an IPython kernel.


(10, 10)

In [42]:
np.sqrt(b.T.dot(YY_coefficient).dot(b)).shape

D:\anaconda\envs\zhu-python\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in sqrt
  """Entry point for launching an IPython kernel.


(10, 10)